# Special Features of the CensusDataframe

Use the ``publicdata.parse_app_url`` to create a new Census URL.


In [1]:
from publicdata import parse_app_url
import pandas as pd
u = parse_app_url('census://2016/5/CA/county/B01001')
df = u.dataframe

## Display

The ``.table`` method has acessors for displaying information about the table, and most importantly, has a pretty display in Jupyter. 

In [2]:
df.table

#,Name,Short Description,Description
0,GEOID,None,GEOID
1,STUSAB,None,STUSAB
2,COUNTY,None,COUNTY
3,NAME,None,NAME
4,B01001_001,Total:,SEX BY AGE for Total Population - Total:
5,B01001_001_m90,None,
6,B01001_002,Male:,SEX BY AGE for Total Population - Male:
7,B01001_002_m90,None,
8,B01001_003,Under 5 years,SEX BY AGE for Total Population - Male: - Under 5 years
9,B01001_003_m90,None,


The ``.titles`` property returns the dataframe with colum names that include the Census column titles, which make it much easier to find the columns you want to use. Combine with ``.head().T`` for an easier-to-read format. 

In [3]:
df.titles.head().T

GEOID,05000US06001,05000US06003,05000US06005,05000US06007,05000US06009
STUSAB,CA,CA,CA,CA,CA
COUNTY,001,003,005,007,009
NAME,"Alameda County, California","Alpine County, California","Amador County, California","Butte County, California","Calaveras County, California"
B01001_001 SEX BY AGE for Total Population - Total:,1605217,1184,36963,223877,44787
B01001_001_m90,0,191,0,0,0
B01001_002 SEX BY AGE for Total Population - Male:,786730,692,19839,110926,22156
B01001_002_m90,51,124,125,147,158
B01001_003 SEX BY AGE for Total Population - Male: - Under 5 years,49865,33,681,6353,750
B01001_003_m90,0,17,80,97,43
B01001_004 SEX BY AGE for Total Population - Male: - 5 to 9 years,50533,34,947,6577,1452


The codes for the column are the first word in the titles. The margin columns are suffixed with '_m90' and are not converted with titles. 

The ``.search_columns`` function returs columns that contain a string or strings, or match a regular expression 


In [4]:
pd.set_option('display.max_colwidth', -1)
df.search_columns('Total').head()

,code,title
0,B01001_001,B01001_001 SEX BY AGE for Total Population - Total:
1,B01001_002,B01001_002 SEX BY AGE for Total Population - Male:
2,B01001_003,B01001_003 SEX BY AGE for Total Population - Male: - Under 5 years
3,B01001_004,B01001_004 SEX BY AGE for Total Population - Male: - 5 to 9 years
4,B01001_005,B01001_005 SEX BY AGE for Total Population - Male: - 10 to 14 years


Or, use a regular expression

In [5]:
import re
df.search_columns(re.compile("Total Pop.*(Female|Male):$")).head()

,code,title
0,B01001_002,B01001_002 SEX BY AGE for Total Population - Male:
1,B01001_026,B01001_026 SEX BY AGE for Total Population - Female:


You can also have more than one term -- they are OR'd -- and mix strings and regexes

In [6]:
import re
df.search_columns('14', re.compile("Total Pop.*(Female|Male):$")).head()

,code,title
0,B01001_005,B01001_005 SEX BY AGE for Total Population - Male: - 10 to 14 years
1,B01001_014,B01001_014 SEX BY AGE for Total Population - Male: - 40 to 44 years
2,B01001_014_m90,B01001_014_m90
3,B01001_029,B01001_029 SEX BY AGE for Total Population - Female: - 10 to 14 years
4,B01001_002,B01001_002 SEX BY AGE for Total Population - Male:


# Math with Margins


The dataframe understands the margins and can perform the correct math for summation and ratios. 

## Summation

The ``.sum_m`` and ``add_sum_m`` return  summation columns and add them to the dataset, respectively. 


First, lets get a subset of the dataframe. 

In [7]:
total_cols = list(df.search_columns(re.compile("Total Pop.*(Female|Male):$")).code)

sdf = df[total_cols].copy()
print(total_cols)
sdf.head()


['B01001_002', 'B01001_026']


,B01001_002,B01001_002_m90,B01001_026,B01001_026_m90
GEOID,,,,
05000US06001,786730,51,818487,51
05000US06003,692,124,492,88
05000US06005,19839,125,17124,125
05000US06007,110926,147,112951,147
05000US06009,22156,158,22631,158


Note that we only request 2 columns, but the indexing also returned the margin columsn, so we got 4. Now we can add the two together. We'll get 2 columns, one for the sum, and one for the margin. 

In [8]:

x = pd.DataFrame()

x['total'], x['total_margin'] = df.sum_m('B01001_002', 'B01001_026')

x.head()

,total,total_margin
GEOID,,
05000US06001,1605217,72.124892
05000US06003,1184,152.052622
05000US06005,36963,176.776695
05000US06007,223877,207.889394
05000US06009,44787,223.445743


The ``.add_sum_m`` does all of the hard work of adding the columns to the dataset, but requires the first parameter to be the name of the resulting column. 

In [9]:
x = sdf.add_sum_m('total', 'B01001_002', 'B01001_026')
sdf.head() # or x, since the add_sum_m returns the dataframe

,B01001_002,B01001_002_m90,B01001_026,B01001_026_m90,total,total_m90
GEOID,,,,,,
05000US06001,786730,51,818487,51,1605217,72.124892
05000US06003,692,124,492,88,1184,152.052622
05000US06005,19839,125,17124,125,36963,176.776695
05000US06007,110926,147,112951,147,223877,207.889394
05000US06009,22156,158,22631,158,44787,223.445743


### Caution with Sums!

There are some interesting problems with sums. Lets look at the sum of Male and Female compared to the total columns


In [10]:
search = df.search_columns(re.compile("Total Pop.*(Female|Male|Total):$"))
search

,code,title
0,B01001_001,B01001_001 SEX BY AGE for Total Population - Total:
1,B01001_002,B01001_002 SEX BY AGE for Total Population - Male:
2,B01001_026,B01001_026 SEX BY AGE for Total Population - Female:


In [11]:
sdf = df[list(search.code)].copy()

sdf.add_sum_m('total', 'B01001_002', 'B01001_026')

sdf['total_ratio'] = sdf.B01001_001 / sdf.total

sdf[['B01001_001','total', 'total_ratio']].head()

,B01001_001,B01001_001_m90,total,total_m90,total_ratio
GEOID,,,,,
05000US06001,1605217,0,1605217,72.124892,1.0
05000US06003,1184,191,1184,152.052622,1.0
05000US06005,36963,0,36963,176.776695,1.0
05000US06007,223877,0,223877,207.889394,1.0
05000US06009,44787,0,44787,223.445743,1.0


Note that the ratio is not 1, meaning that the sum of males and females is not the same as the reported total. The clue as to why is in the margins column for the total, `B01001_001_m90`: it is mostly 0. A value of 0 indicates that these values were taken from a controlled source, usually administrative data. Since the number was not pproduced with samploy, there is no sample error. The male and female values were calculated from samples, so they are different and have an error.


The moral is to always check if the value you want to calculate already exists. 

## Ratios and Proportions

The ``.ratio`` and ``.proportion`` methods are used for divding numbers.

* For ratio: The numerator should not be a subset of the denominator, such as the ratio of males to females. 
* For proportion:  The numerator should be a subset of the denominator,  

So, if you are dividing  males by females, use the ``.ratio`` method. If you are dividing males by the total population, use the ``.proportion`` method. ( The underlying reason for the difference has to do with whether the resulting value can be greater than 1 ) 

In [12]:
sdf.titles.head().T

GEOID,05000US06001,05000US06003,05000US06005,05000US06007,05000US06009
B01001_001 SEX BY AGE for Total Population - Total:,1.605217e+06,1184.000000,36963.000000,223877.000000,44787.000000
B01001_001_m90,0.000000e+00,191.000000,0.000000,0.000000,0.000000
B01001_002 SEX BY AGE for Total Population - Male:,7.867300e+05,692.000000,19839.000000,110926.000000,22156.000000
B01001_002_m90,5.100000e+01,124.000000,125.000000,147.000000,158.000000
B01001_026 SEX BY AGE for Total Population - Female:,8.184870e+05,492.000000,17124.000000,112951.000000,22631.000000
B01001_026_m90,5.100000e+01,88.000000,125.000000,147.000000,158.000000
total,1.605217e+06,1184.000000,36963.000000,223877.000000,44787.000000
total_m90,7.212489e+01,152.052622,176.776695,207.889394,223.445743
total_ratio,1.000000e+00,1.000000,1.000000,1.000000,1.000000


In [13]:
sdf['mf_ratio'], sdf['mf_ratio_m'] = sdf.ratio( 'B01001_002', 'B01001_026') # male/female
sdf[['mf_ratio','mf_ratio_m']] .head()

,mf_ratio,mf_ratio_m
GEOID,,
05000US06001,0.961200,0.000086
05000US06003,1.406504,0.356101
05000US06005,1.158549,0.011172
05000US06007,0.982072,0.001824
05000US06009,0.979011,0.009770


In [14]:
sdf['male_prop'], sdf['male_prop_m90'] = sdf.proportion( 'B01001_002', 'B01001_001') # male/total
sdf.head()

,B01001_001,B01001_001_m90,B01001_002,B01001_002_m90,B01001_026,B01001_026_m90,total,total_m90,total_ratio,mf_ratio,mf_ratio_m,male_prop,male_prop_m90
GEOID,,,,,,,,,,,,,
05000US06001,1605217,0,786730,51,818487,51,1605217,72.124892,1.0,0.961200,0.000086,0.490108,0.000032
05000US06003,1184,191,692,124,492,88,1184,152.052622,1.0,1.406504,0.356101,0.584459,0.045595
05000US06005,36963,0,19839,125,17124,125,36963,176.776695,1.0,1.158549,0.011172,0.536726,0.003382
05000US06007,223877,0,110926,147,112951,147,223877,207.889394,1.0,0.982072,0.001824,0.495477,0.000657
05000US06009,44787,0,22156,158,22631,158,44787,223.445743,1.0,0.979011,0.009770,0.494697,0.003528


## Error Margin Manipulation

The ``.add_rse`` add a relative standard error column


In [16]:
sdf.add_rse('B01001_001').head()

,B01001_001,B01001_001_m90,B01001_002,B01001_002_m90,B01001_026,B01001_026_m90,total,total_m90,total_ratio,mf_ratio,mf_ratio_m,male_prop,male_prop_m90,B01001_001_rse
GEOID,,,,,,,,,,,,,,
05000US06001,1605217,0,786730,51,818487,51,1605217,72.124892,1.0,0.961200,0.000086,0.490108,0.000032,0.000000
05000US06003,1184,191,692,124,492,88,1184,152.052622,1.0,1.406504,0.356101,0.584459,0.045595,9.806539
05000US06005,36963,0,19839,125,17124,125,36963,176.776695,1.0,1.158549,0.011172,0.536726,0.003382,0.000000
05000US06007,223877,0,110926,147,112951,147,223877,207.889394,1.0,0.982072,0.001824,0.495477,0.000657,0.000000
05000US06009,44787,0,22156,158,22631,158,44787,223.445743,1.0,0.979011,0.009770,0.494697,0.003528,0.000000


In [21]:
There are acessors on the estimate colums that will return alternate margins, including: 

* .m90: 90% margins
* .m95: 95% margins
* .m99: 99% margins
* .se: Standard error
* .rse: Relative standard error

SyntaxError: invalid syntax (<ipython-input-21-e659b8777db1>, line 1)

In [23]:
sdf.B01001_026.se.head()

GEOID
05000US06001    31.003040
05000US06003    53.495441
05000US06005    75.987842
05000US06007    89.361702
05000US06009    96.048632
dtype: float64